In [ ]:
import os, pickle, warnings
from tqdm import tqdm
from pprint import pprint

import pandas as pd
import numpy as np

# modeling/experimentation
import mlflow
import skgstat as skg
import gstools as gs
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from pykrige.compat import Krige

# to load parameters of the experiments
from kedro.config import ConfigLoader

# utils
from joblib import Parallel, delayed
from pandas import IndexSlice as idx
from IPython.display import display

# viz
import seaborn as sns
import matplotlib.pyplot as plt

# experiments
from experiments.noaa import NOAAKrigingExperiment
from sklearn.model_selection import ParameterGrid

# local utils and other imports
from spatial_interpolation import data, utils
from spatial_interpolation.utils.modeling import compute_metrics
from spatial_interpolation.utils.experiments import conf 
from spatial_interpolation.data.load_data import get_ml_workspace
from spatial_interpolation.utils import tqdm_joblib
# from spatial_interpolation.visualization import plot_interpolation
from experiments.configs.evaluation import eval_sets as eval_conf

# notebook configuration
import dotenv
warnings.simplefilter(action='ignore', category=FutureWarning)
dotenv.load_dotenv()

In [ ]:
# load data
experiment = NOAAKrigingExperiment("ordinary_kriging_config", n_jobs=20)
config = experiment.get_config()
dataset = data.NDBCDataLoader(**config.data_loading).load()
df, gdf = dataset.buoys_data, dataset.buoys_geo

locations_within_area = gdf.loc[gdf.within(eval_conf.ndbc[config.eval_set].area)].index.get_level_values("buoy_id").unique()
df = df.loc[df.index.get_level_values("buoy_id").isin(locations_within_area)]
gdf = gdf.loc[idx[:, locations_within_area],:]
dataset = data.NDBCData(df, gdf)

train, test = dataset.split_slice(test=eval_conf.ndbc[config.eval_set].eval)
train_df = train.join()
test_df = test.join()

available_time = df.index.get_level_values("time").unique()
time_range = pd.date_range(available_time.min(), available_time.max(), freq="H")

train, test = dataset.split_slice(test=eval_conf.ndbc[config.eval_set].eval)
train_df = train.join()
test_df = test.join()

time_steps = (
    time_range
    .to_series(name="time_step")
    .rename_axis("time")
    .rank(ascending=True)
)

target = config.target
train_by_times = (
    train_df
    .reset_index()
    .rename(columns={"buoy_id": "location_id"})
    .set_index(["time", "location_id"])
    .sort_index()
    .dropna(subset=[target])
    .join(time_steps)
)
test_by_times = (
    test_df
    .reset_index()
    .rename(columns={"buoy_id": "location_id"})
    .set_index(["time", "location_id"])
    .sort_index()
    .dropna(subset=[target])
    .join(time_steps)
)


In [ ]:
time = pd.Timestamp("2020-05-10 05:00") #train_by_times.index.get_level_values(0)[-1]
dims = config.dimensions
delta = pd.Timedelta("3H")#config.delta)
# krige = Krige(method='ordinary', variogram_model='gaussian')
# krige.fit(train_by_times.loc[time,dims].values,y=train_by_times.loc[time,target].values)
# krige.predict(test_by_times.loc[time,dims].values)

In [ ]:
experiment = NOAAKrigingExperiment("ordinary_kriging_config", n_jobs=20)
config = experiment.get_config()
dims = config.dimensions
print(dims)
krige = config.interpolator(
    dimensions=dims,
    **config.interpolator_params
)
krige.fit(train_by_times.loc[time, dims], y=train_by_times.loc[time, target])
krige.predict(test_by_times.loc[time, dims],with_error=False), test_by_times.loc[time, target].values

In [ ]:
model = krige._model()

pos = [train_by_times.loc[time, dim].values for dim in krige.dimensions[:-1]]
t = train_by_times.loc[(time-delta):time, krige.dimensions[-1]].unique() # 10 time steps
st_grid = gs.generate_st_grid(pos, t)
y = train_by_times.loc[(time-delta):time, target].values
bin_center, gamma = gs.vario_estimate(st_grid,y)
_, _, r2 = model.fit_variogram(bin_center, gamma, return_r2=True)

ok = gs.krige.Krige(model= model, cond_pos=st_grid, cond_val=y)

y_pos = [test_by_times.loc[time, dim].values for dim in krige.dimensions[:-1]]
y_t = test_by_times.loc[time, krige.dimensions[-1]].unique() # 10 time steps
y_st_grid = gs.generate_st_grid(y_pos, y_t)
pred, err = ok(y_st_grid)
pred, test_by_times.loc[time, target].values

In [ ]:
model = krige._model()

pos = [train_by_times.loc[(time-delta):time, dim].values for dim in krige.dimensions]
y = train_by_times.loc[(time-delta):time, target].values
bin_center, gamma = gs.vario_estimate(pos,y)
_, _, r2 = model.fit_variogram(bin_center, gamma, return_r2=True)
ok = gs.krige.Krige(model= model, cond_pos=st_grid, cond_val=y)

pos_hat = [test_by_times.loc[time, dim].values for dim in krige.dimensions]
y_hat, err = ok(pos_hat)
y_hat, test_by_times.loc[time, target].values

In [ ]:
experiment = NOAAKrigingExperiment("ok_time_config", n_jobs=20)
config = experiment.get_config()
dims = config.dimensions
print(dims)
# delta = pd.Timedelta(config.delta)
krige = config.interpolator(
    dimensions=dims,
    **config.interpolator_params
)
krige.fit(train_by_times.loc[(time-delta):time, dims], y=train_by_times.loc[(time-delta):time, target])
krige.predict(test_by_times.loc[time, dims],with_error=False), test_by_times.loc[time, target].values

In [ ]:
config = experiment.get_config()
search_params = config.get("search_parameters").to_dict()
parameters_to_search = list(ParameterGrid(search_params))
len(parameters_to_search)

In [ ]:
krige = config.interpolator(
    dimensions=dims,
    **config.interpolator_params
)
krige.fit(train_by_times.loc[time, dims], y=train_by_times.loc[time, target])
krige.predict(test_by_times.loc[time, dims], with_error=False), test_by_times.loc[time, target].values

In [ ]:
params = parameters_to_search[0]

scores = {}
results = []
for model in tqdm(config.search_parameters.model):
    model = model(latlon=True, rescale=gs.EARTH_RADIUS)
    # model = params["model"](latlon=True, rescale=gs.EARTH_RADIUS)
    # num_bins = params["num_bins"]
    # bins = np.arange(params["num_bins"])
    x = train_by_times.loc[time,dims]
    y = train_by_times.loc[time,target]
    lat, lon = x.values[:,1],x.values[:,0]
    bin_center, gamma = gs.vario_estimate((x.values[:,1],x.values[:,0]),train_by_times.loc[time,target].values, latlon=True)
    _, __, r2 = model.fit_variogram(bin_center, gamma, return_r2=True)
    # krige = gs.krige.Krige(model=model, x.values, y.values, latlon=True)
    scores[f"{model.__class__.__name__}"] = r2
    # if params["num_bins"] == 60:
    fig, ax = plt.subplots(1,1,figsize=(10,10))
    ax.scatter(bin_center, gamma, color="k", label="data")
    model.plot("vario_yadrenko", ax=ax, x_max=bin_center[-1]+np.std(bin_center))
    ax.set_title(f"{model.__class__.__name__} Variogram R2={r2:.2f}")

    krige = gs.krige.Krige(model=model, cond_pos=(lat, lon), cond_val=y.values)
    lat_tst, lon_tst = test_by_times.loc[time,dims].values[:,1],test_by_times.loc[time,dims].values[:,0]
    y_pred, err = krige((lat_tst, lon_tst))
    metrics = compute_metrics(y_pred, test_by_times.loc[time,target].values)
    metrics["model"] = model.__class__.__name__
    results.append(metrics)

results_df = pd.DataFrame(results).sort_values("rmse",ascending=True).set_index("model").round(3)
print("Best models (sorted):\n",sorted(scores,key=scores.get, reverse=True))
results_df

In [ ]:
plt.set_cmap('jet')
x = train_by_times.loc[time,dims]
lat, lon = x.values[:,1],x.values[:,0]
y = train_by_times.loc[time,target]
bins = gs.standard_bins((lon,lat),latlon=True)
bin_c, vario = gs.vario_estimate((x.values[:,1],x.values[:,0]),train_by_times.loc[time,target].values,latlon=True)
model = gs.Linear(latlon=True, rescale=gs.EARTH_RADIUS)
model.fit_variogram(bin_c, vario)
ax = model.plot("vario_yadrenko")#, x_max=bins[-1])
ax.scatter(bin_c, vario)
ax.set(xlabel="Distance", ylabel="Spatial Variance", title=f"{model.__class__.__name__} Variogram")
print(model)
uk = gs.krige.Ordinary(
    model=model,
    cond_pos=(x.values[:,1],x.values[:,0]),
    cond_val=train_by_times.loc[time,target].values,
    # drift_functions=north_south_drift,
)
xmin, ymin, xmax, ymax = area.bounds
grid_lats, grid_lons = np.linspace(ymin, ymax, 100), np.linspace(xmin, xmax, 100)
uk.set_pos((grid_lats, grid_lons), mesh_type="structured")
uk(return_var=False, store="field")
uk(only_mean=True, store="mean_field")

levels = np.linspace(0, 4, 64)
fig, ax = plt.subplots(1, 3, figsize=(12, 8), sharey=True)
sca = ax[0].scatter(lon, lat, c=y, vmin=0, vmax=4, s=10)
co1 = ax[1].contourf(grid_lons, grid_lats, uk["field"], levels)
co2 = ax[2].contourf(grid_lons, grid_lats, uk["mean_field"], levels)

[df_countries.plot(ax=ax[i],alpha=0.5) for i in range(3)]
[ax[i].set_xlim([xmin, xmax]) for i in range(3)]
[ax[i].set_ylim([ymin, ymax]) for i in range(3)]
[ax[i].set_xlabel("Lon in deg") for i in range(3)]
ax[0].set_ylabel("Lat in deg")

ax[0].set_title(f"Wave Height observations\n({pd.Timestamp(time):%b %d, %Y at %H:%M})")
ax[1].set_title("Kriging Interpolated\nvalues")
ax[2].set_title("Estimated mean\n")

fmt = dict(orientation="horizontal", shrink=0.5, fraction=0.1, pad=0.1)
fig.colorbar(co2, ax=ax, **fmt).set_label("Wave height in meters")
# fig.tight_layout()

In [ ]:
pred, ss = krige.model.execute(
    "points", 
    test_by_times.loc[time,dims].values[:,0],test_by_times.loc[time,dims].values[:,1]
)
pd.DataFrame({target:pred},index=test_by_times.loc[time].index)\
    .join(test_by_times[target],rsuffix="_true")\
        .rename(columns={target:"y_pred", f"{target}_true":"y_true"})

In [ ]:
V = skg.Variogram(train_by_times.loc[time,dims], train_by_times.loc[time,target])
# V.model = "gaussian"
print("Variogram fitted:",V.model.__name__)
kr = V.to_gs_krige()
kr.set_pos((test_by_times.loc[time,dims].values[:,0],test_by_times.loc[time,dims].values[:,1]),mesh_type="unstructured")
kr(return_var=False, store="field")
kr(only_mean=True, store="mean_field")
pred= kr["field"]
pd.DataFrame({target:pred},index=test_by_times.loc[time].index)\
    .join(test_by_times[target],rsuffix="_true")\
        .rename(columns={target:"y_pred", f"{target}_true":"y_true"})

In [ ]:
from skgstat import OrdinaryKriging

ok = OrdinaryKriging(V, min_points=5, max_points=20, mode='exact')
ok.transform(test_by_times.loc[time,dims].values)